In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import os
import pandas as pd
import pickle

In [ ]:
def get_scardec_stf(scardec_name, wanted_type = 'fctopt'):
    time = []
    momentrate = []

    event = os.listdir(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}')
    starts = [n for n, l in enumerate(event) if l.startswith(wanted_type)]
    with open(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}/{event[starts[0]]}') as f:
        lines = f.read().splitlines()

    lines = lines[2:]
    for line in lines:
        split = line.split(' ')
        split = [s for s in split if s not in ['', ' ', '\n']]
        time.append(float(split[0]))
        momentrate.append(float(split[1]))

    momentrate = np.array(momentrate)
    time = np.array(time)
    return momentrate, time

In [ ]:
def get_ye_stf(ye_name):
    data_path = '/home/earthquakes1/homes/Rebecca/phd/stf/data/Ye_et_al_2016/'
    momentrate = []
    time = []

    with open(data_path + str(ye_name), 'r') as f:
        data = f.readlines()
        for line in data:
            line = line.strip()
            line = line.rstrip()
            if line[0] not in ['0','1','2','3','4','5','6','7','8','9']:
                continue
            line = line.split()
            time.append(float(line[0]))
            momentrate.append(float(line[1]))
    momentrate = np.array(momentrate)
    time = np.array(time)
    return momentrate, time

In [ ]:
def get_usgs_stf(usgs_name):
    data_path = '/home/earthquakes1/homes/Rebecca/phd/stf/data/USGS/'
    momentrate = []
    time = []

    with open(data_path + str(usgs_name), 'r') as f:
        data = f.readlines()
        for line in data:
            line = line.strip()
            line = line.rstrip()
            if line[0] not in ['0','1','2','3','4','5','6','7','8','9']:
                continue
            line = line.split()
            time.append(float(line[0]))
            momentrate.append(float(line[1]))

    momentrate = np.array(momentrate)
    time = np.array(time)

    momentrate = momentrate / 10**7 # convert to Nm from dyne cm
    return momentrate, time

In [ ]:
def get_sigloch_stf(sigloch_name):
    data_path = '/home/siglochnas1/shared/AmplitudeProjects/pdata_processed/psdata_events/'
    momentrate = []
    time = []

    file_path = data_path + str(sigloch_name) + '/outfiles/ampinv.stf.xy'

    with open(file_path, 'r') as file:
        content = file.read()
        content = content.split('\n')
        greater_than_count = content.count('>')
        if greater_than_count > 0:
            time = [[]]
            momentrate = [[]]
            for i in range(greater_than_count-1):
                time.append([])
                momentrate.append([])


        stf_count = 0
        for c in content:
            if c not in ['<', '>', '']:
                split = c.split()
                time[stf_count].append(float(split[0]))
                momentrate[stf_count].append(float(split[1]))
            else:
                stf_count += 1
    return momentrate, time

In [ ]:
def load_isc(isc_name):
    isc_save_path = '/home/earthquakes1/homes/Rebecca/phd/stf/data/isc/'
    with open(f'{isc_save_path}{isc_name}/{isc_name}.txt', 'rb') as f:
        stf_list = pickle.load(f)
    with open(f'{isc_save_path}{isc_name}/{isc_name}_norm_info.txt', 'rb') as f:
        norm_dict = pickle.load(f)

    time = np.arange(0, 25.6, 0.1)
    momentrate = np.array(stf_list)*norm_dict['mo_norm']*10**8,
    return momentrate, time

In [ ]:
combined = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined_scardec_ye_usgs_sigloch_isc_mag.csv')

In [ ]:
combined.columns = ['event', 'scardec', 'ye', 'isc', 'sigloch', 'usgs', 'mag']

In [ ]:
def plot_stf(ax, time, momentrate, detected_end_time, linecolor = 'k', linestyle = '--', label_str = '', plot_bar = True):
    ax.plot(time, momentrate, color=linecolor, label = label_str + ' STF' )
    if plot_bar is True:
        ax.axhline(max(momentrate)/10, color='#ffa600')
        ax.fill_between(time, 0, 10*max(momentrate)/100, facecolor='#ffa600', alpha=0.1)

    ax.fill_between(time, momentrate, 0, where=momentrate>0, facecolor=linecolor, alpha=0.05)

    # ax.axvline(detected_end_time, color=linecolor, linestyle=linestyle, label = label_str + ' (3) end')

    ax.set_ylim(0, 1.1*max(momentrate))
    ax.set_xlim(0, max(time))

    ax.legend()

    return ax


In [ ]:
def find_end_stf(momentrate, time, threshold = 0.5):
    not_zero = np.where(momentrate > 0)[0]
    #print(max(momentrate))
    start = min(not_zero)
    end = max(not_zero)

    detected_end = end
    detected_end_time = time[end]

    time = time[:end]
    momentrate = momentrate[:end]

    less_than_10 = np.where(momentrate <= 10*max(momentrate)/100)[0]
    #print(less_than_10)
    print(start)
    total_moment = scipy.integrate.simpson(momentrate[start:end],
                                        dx = time[start+1]-time[start])
    #print(less_than_10)
    for i in less_than_10:
        if i <= start:
            continue
        if i == 0:
            continue
        moment = scipy.integrate.simpson(momentrate[start:i],
                                        dx = time[start+1]-time[start])
        #print(i, moment/total_moment)
        if moment >= threshold * total_moment:
            #print('inif')
            #print(f'first time where < 10% of total momentrate and 50% of moment released: {time[i]} s')
            detected_end_time = time[i]
            detected_end = i
            #print(f'proportion of moment released: {(moment/total_moment)*100:.2f}%')
            break
    return detected_end_time, detected_end, time[start], start
    #return time[end], end

In [ ]:
def find_99_end_stf(momentrate, time):
    not_zero = np.where(momentrate > 0)[0]
    #print(max(momentrate))
    start = min(not_zero)
    end = max(not_zero)

    detected_end = end
    detected_end_time = time[end]

    time = time[:end]
    momentrate = momentrate[:end]

    less_than_10 = np.where(momentrate <= 10*max(momentrate)/100)[0]
    print(less_than_10)
    print(start)
    total_moment = scipy.integrate.simpson(momentrate[start:end],
                                        dx = time[start+1]-time[start])
    #print(less_than_10)
    for i in less_than_10:
        if i <= start:
            continue
        if i == 0:
            continue
        moment = scipy.integrate.simpson(momentrate[start:i],
                                        dx = time[start+1]-time[start])
        #print(i, moment/total_moment)
        if moment >= 0.99 * total_moment:
            #print('inif')
            #print(f'first time where < 10% of total momentrate and 50% of moment released: {time[i]} s')
            detected_end_time = time[i]
            detected_end = i
            #print(f'proportion of moment released: {(moment/total_moment)*100:.2f}%')
            break

    return detected_end_time, detected_end, time[start], start
    #return time[end], end

In [ ]:
def find_zero_end_stf(momentrate, time):
    not_zero = np.where(momentrate > 0)[0]
    #print(max(momentrate))
    start = min(not_zero)
    end = max(not_zero)

    detected_end = end
    detected_end_time = time[end]

    return detected_end_time, detected_end, time[start], start
    #return time[end], end

In [ ]:
colors = ['#003f5c', '#bc5090', '#ef5675', 'g', 'c', 'm', 'k', 'y']
styles = ['--', '-.', ':', '--', '-.', ':', '--', '-.']
eventcount = 0
already_made = os.listdir('/home/earthquakes1/homes/Rebecca/phd/stf/figures/ends/all_on_one/both/')
for i, row in combined.iterrows():
    # if row.event != '20200623_1529':
    #     continue
    # if row.sigloch != '0706.2011.026.a':
    #     continue
    if i > 100:
        continue
    # if row.event + '.png' in already_made:
    #     continue
    lines_count = 0
    count = 0
    print(row)
    sources = 0
    for r in row[1:-1]:
        if r not in [0, '0']:
            sources += 1

    max_end = 0
    y_max = 0
    max_end = 0

    fig, axs = plt.subplots(1, 1, figsize=(10, 5))
    axs = [axs]

    ends_different = False
    count_datasets = 0
    if row.scardec not in ['0', 0]:
        momentrate, time = get_scardec_stf(row['scardec'], wanted_type = 'fctmoy')
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        axs[count] = plot_stf(axs[count], time, momentrate, detected_end_time, colors[lines_count], '--', label_str = 'SCARDEC Average', plot_bar = False)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'SCARDEC Average 99 end', linewidth = 2)
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'SCARDEC Average Zero end', linewidth = 2)
        max_end = max(max_end, max(time))
        y_max = max(y_max, max(momentrate))
        lines_count += 1

        momentrate, time = get_scardec_stf(row['scardec'], wanted_type = 'fctopt')
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        axs[count] = plot_stf(axs[count], time, momentrate, detected_end_time, colors[lines_count], '--', label_str = 'SCARDEC Optimal', plot_bar = False)
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'SCARDEC Optimal zero end', linewidth = 2)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'SCARDEC Optimal 99 end', linewidth = 2)
        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.ye not in ['0', 0]:
        momentrate, time = get_ye_stf(row['ye'])
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'Ye 99 end', linewidth = 2)

        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'Ye zero end')
        axs[count] = plot_stf(axs[count], time, momentrate, detected_end_time, colors[lines_count], '--', label_str = 'Ye', plot_bar = False)
        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.usgs not in ['0', 0]:
        momentrate, time = get_usgs_stf(row['usgs'])
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'USGS 99 end', linewidth = 2)

        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'USGS zero end')
        axs[count] = plot_stf(axs[count], time, momentrate, detected_end_time, colors[lines_count], '--', label_str = 'USGS', plot_bar = False)
        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.isc not in ['0', 0]:
        momentrate, time = load_isc(row['isc'])
        momentrate = momentrate[0]
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'ISC  99 end', linewidth = 2)
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label =  'ISC zero end')
        axs[count] = plot_stf(axs[count], time, momentrate, detected_end_time, colors[lines_count], '--', label_str = 'ISC', plot_bar = False)
        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.sigloch not in ['0', 0]:
        momentrate_list, time_list = get_sigloch_stf(row['sigloch'])
        for momentrate, time in zip(momentrate_list, time_list):
            momentrate = np.array(momentrate)
            y_max = max(y_max, 10**max(momentrate))
            time = np.arange(0, 25.6, 0.1)#np.array(time)
            detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(10**momentrate, time)
            time = time - detected_start_time # shift to start STF at zero
            detected_end_time = detected_end_time - detected_start_time
            #print(detected_end_time)
            max_end = max(detected_end_time, max_end)
            axs[count] = plot_stf(axs[count], time, 10**momentrate, detected_end_time, colors[lines_count], '--', label_str = 'Sigloch ', plot_bar = False)
            detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(10**momentrate, time)
            #print(detected_zero_end_time)
            #max_end = max(max_end, detected_zero_end_time)
            axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'Sigloch zero end', linewidth = 2)
            detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(10**momentrate, time)
            #print(detected_zero_end_time)
            max_end = max(max_end, detected_99_end_time)
            axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'Sigloch 99 end', linewidth = 2)
            if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
                ends_different = True
            lines_count += 1
        count_datasets += 1

    for ax in axs:
        ax.set_xlim(0, 1.1*max_end)
        ax.set_ylim(0, 1.1*y_max)
        if np.isnan(row.mag) == False:
            ax.set_title(row.event + f' | Magnitude: {row.mag}')
        else:
            ax.set_title(row.event)
    plt.legend()
    if count_datasets >= 2:
        if ends_different is True:
            plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/ends/all_on_one/end_comparisons/different/{row.event}.png')
        else:
            plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/ends/all_on_one/end_comparisons/same/{row.event}.png')
        # if row.isc not in ['0', 0]:
    #     plt.show()
    #     break
    # else:
    #     plt.close()
    #     continue
    #axs[0].set_yscale('log')
    #plt.close()
    print(max_end)
    plt.show()






In [ ]:
colors = ['#003f5c', '#bc5090', '#ef5675', 'g', 'c', 'm', 'k', 'y']
styles = ['--', '-.', ':', '--', '-.', ':', '--', '-.']
eventcount = 0
already_made = os.listdir('/home/earthquakes1/homes/Rebecca/phd/stf/figures/ends/all_on_one/both/')
for i, row in combined.iterrows():
    # if row.event != '20200623_1529':
    #     continue
    # if row.sigloch != '0706.2011.026.a':
    #     continue
    if i > 100:
        continue
    # if row.event + '.png' in already_made:
    #     continue
    lines_count = 0
    count = 0
    print(row)
    sources = 0
    for r in row[1:-1]:
        if r not in [0, '0']:
            sources += 1

    max_end = 0
    y_max = 0
    max_end = 0

    fig, axs = plt.subplots(1, 1, figsize=(10, 5))
    axs = [axs]

    ends_different = False
    count_datasets = 0
    if row.scardec not in ['0', 0]:
        momentrate, time = get_scardec_stf(row['scardec'], wanted_type = 'fctmoy')
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        axs[count] = plot_stf(axs[count], time, momentrate/max(momentrate), detected_end_time, colors[lines_count], '--', label_str = 'SCARDEC Average', plot_bar = False)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'SCARDEC Average 99 end', linewidth = 2)
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'SCARDEC Average Zero end', linewidth = 2)
        max_end = max(max_end, max(time))
        y_max = max(y_max, max(momentrate))
        lines_count += 1

        momentrate, time = get_scardec_stf(row['scardec'], wanted_type = 'fctopt')
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        axs[count] = plot_stf(axs[count], time, momentrate/max(momentrate), detected_end_time, colors[lines_count], '--', label_str = 'SCARDEC Optimal', plot_bar = False)
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'SCARDEC Optimal zero end', linewidth = 2)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'SCARDEC Optimal 99 end', linewidth = 2)
        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.ye not in ['0', 0]:
        momentrate, time = get_ye_stf(row['ye'])
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'Ye 99 end', linewidth = 2)

        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'Ye zero end')
        axs[count] = plot_stf(axs[count], time, momentrate/max(momentrate), detected_end_time, colors[lines_count], '--', label_str = 'Ye', plot_bar = False)
        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.usgs not in ['0', 0]:
        momentrate, time = get_usgs_stf(row['usgs'])
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'USGS 99 end', linewidth = 2)

        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'USGS zero end')
        axs[count] = plot_stf(axs[count], time, momentrate/max(momentrate), detected_end_time, colors[lines_count], '--', label_str = 'USGS', plot_bar = False)
        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.isc not in ['0', 0]:
        momentrate, time = load_isc(row['isc'])
        momentrate = momentrate[0]
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'ISC  99 end', linewidth = 2)
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label =  'ISC zero end')
        axs[count] = plot_stf(axs[count], time, momentrate/max(momentrate), detected_end_time, colors[lines_count], '--', label_str = 'ISC', plot_bar = False)
        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.sigloch not in ['0', 0]:
        momentrate_list, time_list = get_sigloch_stf(row['sigloch'])
        for momentrate, time in zip(momentrate_list, time_list):
            momentrate = np.array(momentrate)
            y_max = max(y_max, 10**max(momentrate))
            time = np.arange(0, 25.6, 0.1)#np.array(time)
            detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(10**momentrate, time)
            time = time - detected_start_time # shift to start STF at zero
            detected_end_time = detected_end_time - detected_start_time
            #print(detected_end_time)
            max_end = max(detected_end_time, max_end)
            detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(10**momentrate, time)
            #print(detected_zero_end_time)
            #max_end = max(max_end, detected_zero_end_time)
            axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'Sigloch zero end', linewidth = 2)
            detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(10**momentrate, time)
            #print(detected_zero_end_time)
            max_end = max(max_end, detected_99_end_time)
            axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'Sigloch 99 end', linewidth = 2)
            if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
                ends_different = True
            axs[count] = plot_stf(axs[count], time, (10**momentrate)/max(10**momentrate), detected_end_time, colors[lines_count], '--', label_str = 'Sigloch ', plot_bar = False)
            lines_count += 1
        count_datasets += 1

    for ax in axs:
        ax.set_xlim(0, 1.1*max_end)
        #ax.set_ylim(0, 1.1*y_max)
        if np.isnan(row.mag) == False:
            ax.set_title(row.event + f' | Magnitude: {row.mag}')
        else:
            ax.set_title(row.event)
    plt.legend()
    if count_datasets >= 2:
        if ends_different is True:
            plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/ends/all_on_one/norm/end_comparisons/different/{row.event}.png')
        else:
            plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/ends/all_on_one/norm/end_comparisons/same/{row.event}.png')
        # if row.isc not in ['0', 0]:
    #     plt.show()
    #     break
    # else:
    #     plt.close()
    #     continue
    #axs[0].set_yscale('log')
    #plt.close()
    print(max_end)
    plt.show()






In [ ]:
colors = ['#003f5c', '#bc5090', '#ef5675', 'g', 'c', 'm', 'k', 'y']
styles = ['--', '-.', ':', '--', '-.', ':', '--', '-.']
eventcount = 0
already_made = os.listdir('/home/earthquakes1/homes/Rebecca/phd/stf/figures/ends/all_on_one/both/')
for i, row in combined.iterrows():
    if row.event not in ['20070308_0503', '20080106_0514', '20120930_1631', '20161225_1422', '20100205_0659']:
        continue
    # if row.event != '20200623_1529':
    #     continue
    # if row.sigloch != '0706.2011.026.a':
    #     continue
    if i > 100:
        continue
    # if row.event + '.png' in already_made:
    #     continue
    lines_count = 0
    count = 0
    print(row)
    sources = 0
    for r in row[1:-1]:
        if r not in [0, '0']:
            sources += 1
    if row['scardec'] not in ['0', 0]:
        sources += 1

    max_end = 0
    y_max = 0
    max_end = 0

    fig, axs = plt.subplots(sources, 1, figsize=(6, 8), sharex = True)
    if sources == 1:
        axs = [axs]

    ends_different = False
    count_datasets = 0
    if row.scardec not in ['0', 0]:
        momentrate, time = get_scardec_stf(row['scardec'], wanted_type = 'fctmoy')
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        axs[count] = plot_stf(axs[count], time, momentrate, detected_end_time, colors[lines_count], '--', label_str = 'SCARDEC Average', plot_bar = False)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'SCARDEC Average 99 end', linewidth = 2)
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'SCARDEC Average Zero end', linewidth = 2)
        max_end = max(max_end, max(time))
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        count += 1

        momentrate, time = get_scardec_stf(row['scardec'], wanted_type = 'fctopt')
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        axs[count] = plot_stf(axs[count], time, momentrate, detected_end_time, colors[lines_count], '--', label_str = 'SCARDEC Optimal', plot_bar = False)
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'SCARDEC Optimal zero end', linewidth = 2)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'SCARDEC Optimal 99 end', linewidth = 2)
        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.ye not in ['0', 0]:
        momentrate, time = get_ye_stf(row['ye'])
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'Ye 99 end', linewidth = 2)

        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'Ye zero end')
        axs[count] = plot_stf(axs[count], time, momentrate, detected_end_time, colors[lines_count], '--', label_str = 'Ye', plot_bar = False)
        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.usgs not in ['0', 0]:
        momentrate, time = get_usgs_stf(row['usgs'])
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'USGS 99 end', linewidth = 2)

        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'USGS zero end')
        axs[count] = plot_stf(axs[count], time, momentrate, detected_end_time, colors[lines_count], '--', label_str = 'USGS', plot_bar = False)
        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.isc not in ['0', 0]:
        momentrate, time = load_isc(row['isc'])
        momentrate = momentrate[0]
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'ISC  99 end', linewidth = 2)
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label =  'ISC zero end')
        axs[count] = plot_stf(axs[count], time, momentrate, detected_end_time, colors[lines_count], '--', label_str = 'ISC', plot_bar = False)
        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.sigloch not in ['0', 0]:
        momentrate_list, time_list = get_sigloch_stf(row['sigloch'])
        for momentrate, time in zip(momentrate_list, time_list):
            momentrate = np.array(momentrate)
            y_max = max(y_max, 10**max(momentrate))
            time = np.arange(0, 25.6, 0.1)#np.array(time)
            detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(10**momentrate, time)
            time = time - detected_start_time # shift to start STF at zero
            detected_end_time = detected_end_time - detected_start_time
            #print(detected_end_time)
            max_end = max(detected_end_time, max_end)
            axs[count] = plot_stf(axs[count], time, 10**momentrate, detected_end_time, colors[lines_count], '--', label_str = 'Sigloch ', plot_bar = False)
            detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(10**momentrate, time)
            #print(detected_zero_end_time)
            #max_end = max(max_end, detected_zero_end_time)
            axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'Sigloch zero end', linewidth = 2)
            detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(10**momentrate, time)
            #print(detected_zero_end_time)
            max_end = max(max_end, detected_99_end_time)
            axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'Sigloch 99 end', linewidth = 2)
            count += 1
            if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
                ends_different = True
            lines_count += 1
        count_datasets += 1

    for ax in axs:
        ax.set_xlim(0, 1.1*max_end)
        ax.set_ylim(0, 1.1*y_max)
        ax.set_ylabel('Moment rate (Nm/s)')
    axs[-1].set_xlabel('Time (s)')
    if np.isnan(row.mag) is False:
        fig.suptitle(row.event + f' | Magnitude: {row.mag}')
    else:
        fig.suptitle(row.event)
    plt.legend()

    plt.tight_layout()

    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/ends/for_paper/{row.event}.png')
    # if count_datasets >= 2:
    #     if ends_different is True:
    #         plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/ends/all_on_one/end_comparisons/different/{row.event}.png')
    #     else:
    #         plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/ends/all_on_one/end_comparisons/same/{row.event}.png')
    #     # if row.isc not in ['0', 0]:
    # #     plt.show()
    #     break
    # else:
    #     plt.close()
    #     continue
    #axs[0].set_yscale('log')
    #plt.close()
    print(max_end)
    plt.show()






In [ ]:
colors = ['#003f5c', '#bc5090', '#ef5675', 'g', 'c', 'm', 'k', 'y']
styles = ['--', '-.', ':', '--', '-.', ':', '--', '-.']
eventcount = 0
already_made = os.listdir('/home/earthquakes1/homes/Rebecca/phd/stf/figures/ends/all_on_one/both/')
for i, row in combined.iterrows():
    if row.event not in ['20070308_0503', '20080106_0514', '20120930_1631', '20161225_1422', '20100205_0659']:
        continue
    # if row.event != '20200623_1529':
    #     continue
    # if row.sigloch != '0706.2011.026.a':
    #     continue
    if i > 100:
        continue
    # if row.event + '.png' in already_made:
    #     continue
    lines_count = 0
    count = 0
    print(row)
    sources = 0
    for r in row[1:-1]:
        if r not in [0, '0']:
            sources += 1
    if row['scardec'] not in ['0', 0]:
        sources += 1

    max_end = 0
    y_max = 0
    max_end = 0
    if sources < 4:
        fig, axs = plt.subplots(sources, 1, figsize=(6, 2.5*sources), sharex = True)
    else:
        fig, axs = plt.subplots(sources, 1, figsize=(6, 8), sharex = True)
    if sources == 1:
        axs = [axs]

    ends_different = False
    count_datasets = 0
    if row.scardec not in ['0', 0]:
        momentrate, time = get_scardec_stf(row['scardec'], wanted_type = 'fctmoy')
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        axs[count] = plot_stf(axs[count], time, momentrate/max(momentrate), detected_end_time, colors[lines_count], '--', label_str = 'SCARDEC Average', plot_bar = False)
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'SCARDEC Average (1) end', linewidth = 2)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'SCARDEC Average (2) end', linewidth = 2)
        axs[count].axvline(detected_end_time, color=colors[lines_count], linestyle='--', label = 'SCARDEC Average (3) end', linewidth = 2)
        max_end = max(max_end, max(time))
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        count += 1

        momentrate, time = get_scardec_stf(row['scardec'], wanted_type = 'fctopt')
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        axs[count] = plot_stf(axs[count], time, momentrate/max(momentrate), detected_end_time, colors[lines_count], '--', label_str = 'SCARDEC Optimal', plot_bar = False)
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'SCARDEC Optimal (1) end', linewidth = 2)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'SCARDEC Optimal (2) end', linewidth = 2)
        axs[count].axvline(detected_end_time, color=colors[lines_count], linestyle='--', label = 'SCARDEC Optimal (3) end', linewidth = 2)
        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.ye not in ['0', 0]:
        momentrate, time = get_ye_stf(row['ye'])
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count] = plot_stf(axs[count], time, momentrate/max(momentrate), detected_end_time, colors[lines_count], '--', label_str = 'Ye', plot_bar = False)
        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'Ye (1) end', linewidth = 2)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'Ye (2) end', linewidth = 2)
        axs[count].axvline(detected_end_time, color=colors[lines_count], linestyle='--', label = 'Ye (3) end', linewidth = 2)

        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.usgs not in ['0', 0]:
        momentrate, time = get_usgs_stf(row['usgs'])
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count] = plot_stf(axs[count], time, momentrate/max(momentrate), detected_end_time, colors[lines_count], '--', label_str = 'USGS', plot_bar = False)
        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'USGS (1) end', linewidth = 2)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'USGS (2) end', linewidth = 2)
        axs[count].axvline(detected_end_time, color=colors[lines_count], linestyle='--', label = 'USGS (3) end', linewidth = 2)

        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.isc not in ['0', 0]:
        momentrate, time = load_isc(row['isc'])
        momentrate = momentrate[0]
        detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time)
        time = time - detected_start_time # shift to start STF at zero
        detected_end_time = detected_end_time - detected_start_time
        detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(momentrate, time)
        axs[count] = plot_stf(axs[count], time, momentrate/max(momentrate), detected_end_time, colors[lines_count], '--', label_str = 'ISC', plot_bar = False)
        axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label =  'ISC (1) end', linewidth = 2)
        detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(momentrate, time)
        axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'ISC (2) end', linewidth = 2)
        axs[count].axvline(detected_end_time, color=colors[lines_count], linestyle='--', label = 'ISC (3) end', linewidth = 2)
        max_end = max(max_end, detected_end_time)
        #max_end = max(max_end, detected_zero_end_time)
        max_end = max(max_end, detected_99_end_time)
        y_max = max(y_max, max(momentrate))
        lines_count += 1
        count += 1
        if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
            ends_different = True
        count_datasets += 1
    if row.sigloch not in ['0', 0]:
        momentrate_list, time_list = get_sigloch_stf(row['sigloch'])
        for momentrate, time in zip(momentrate_list, time_list):
            momentrate = np.array(momentrate)
            y_max = max(y_max, 10**max(momentrate))
            time = np.arange(0, 25.6, 0.1)#np.array(time)
            detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(10**momentrate, time)
            time = time - detected_start_time # shift to start STF at zero
            detected_end_time = detected_end_time - detected_start_time
            #print(detected_end_time)
            max_end = max(detected_end_time, max_end)
            axs[count] = plot_stf(axs[count], time, 10**momentrate/10**max(momentrate), detected_end_time, colors[lines_count], '--', label_str = 'Sigloch ', plot_bar = False)
            detected_zero_end_time, detected_zero_end, detected_zero_start_time, detected_zero_start = find_zero_end_stf(10**momentrate, time)
            #print(detected_zero_end_time)
            #max_end = max(max_end, detected_zero_end_time)
            axs[count].axvline(detected_zero_end_time, color=colors[lines_count], linestyle=':', label = 'Sigloch (1) end', linewidth = 2)
            detected_99_end_time, detected_99_end, detected_99_start_time, detected_99_start = find_99_end_stf(10**momentrate, time)
            #print(detected_zero_end_time)
            max_end = max(max_end, detected_99_end_time)
            axs[count].axvline(detected_99_end_time, color=colors[lines_count], linestyle='-.', label = 'Sigloch (2) end', linewidth = 2)
            axs[count].axvline(detected_end_time, color=colors[lines_count], linestyle='--', label = 'Sigloch (3) end', linewidth = 2)
            count += 1
            if 0.9 * detected_99_end_time < detected_end_time < 1.1 * detected_99_end_time:
                ends_different = True
            lines_count += 1
        count_datasets += 1

    for ax in axs:
        ax.set_xlim(0, 1.1*max_end)
        ax.set_ylim(0, 1.1)#*y_max)
        ax.legend(loc='upper right')
        ax.set_ylabel('Normalised moment rate')
    axs[-1].set_xlabel('Time (s)')
    # if np.isnan(row.mag) is False:
    #     fig.suptitle(row.event + f' | Magnitude: {row.mag}')
    # else:
    #     fig.suptitle(row.event)
    #plt.legend()

    plt.tight_layout()

    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/ends/for_paper/70_threshold/{row.event}_norm.png', bbox_inches='tight')
    # if count_datasets >= 2:
    #     if ends_different is True:
    #         plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/ends/all_on_one/end_comparisons/different/{row.event}.png')
    #     else:
    #         plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/ends/all_on_one/end_comparisons/same/{row.event}.png')
    #     # if row.isc not in ['0', 0]:
    # #     plt.show()
    #     break
    # else:
    #     plt.close()
    #     continue
    #axs[0].set_yscale('log')
    #plt.close()
    print(max_end)
    plt.show()




